###############################################################################
# SRSWTISummarizer
###############################################################################


In [ ]:
# --------------------------- Imports ------------------------------
from srswti_axis import SRSWTISummarizer
from utils import * 

/home/zhreyas/miniconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-10 20:44:47.583231: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741619687.603537 1105640 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741619687.609297 1105640 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 20:44:47.630391: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-c

# --------------------------- Basic Demos ------------------------


In [ ]:
# Initialize the summarizer
summarizer = SRSWTISummarizer()

In [ ]:
# Define a sample text about deep neural networks for summarization
long_text_1 = """
Deep neural networks have achieved state-of-the-art performance in multiple tasks, 
but they often require large datasets and extensive computation. Researchers 
are investigating more efficient architectures and training methodologies.
"""

# Use the previously initialized SRSWTISummarizer to summarize the text
# Parameters:
# - model_key="SRSWTI-LW1": Specifies which model variant to use for summarization
# - min_length=20: Sets a minimum length constraint for the generated summary
# - max_length=60: Sets a maximum length constraint for the generated summary
summary_1 = summarizer.summarize_text(long_text_1, model_key="SRSWTI-LW1", min_length=20, max_length=60)

# Print a header for this summarization example
print("== Summarization 1 ==")
# Print the generated summary followed by a blank line
print(summary_1, "\n")


[SRSWTI INFO] 2025-03-10 20:53:00 - Starting single text summarization with SRSWTI-LW1
[SRSWTI INFO] 2025-03-10 20:53:00 - Initializing SRSWTI-LW1 summarization engine on device 0...


Device set to use cuda:0
Your max_length is set to 60, but your input_length is only 42. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


[SRSWTI INFO] 2025-03-10 20:57:29 - Engine SRSWTI-LW1 initialized successfully
[SRSWTI METRICS] Operation: Single text summarization - Duration: 270.22 seconds
== Summarization 1 ==
deep neural networks have achieved state-of-the-art performance in multiple tasks . they often require large datasets and extensive computation . 



In [8]:
batch_texts_2 = [
    "Paragraph about climate change impacts on agriculture...",
    "Another paragraph about carbon emissions and policy changes..."
]
summary_2 = summarizer.summarize_batch(batch_texts_2, model_key="SRSWTI-LW1", max_length=50)
print("== Summarization 2 ==")
print(summary_2, "\n")

Your max_length is set to 50, but your input_length is only 12. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)


[SRSWTI INFO] 2025-03-10 20:57:30 - Starting batch summarization with SRSWTI-LW1
[SRSWTI METRICS] Operation: Batch summarization - Duration: 0.67 seconds
== Summarization 2 ==
["paragraph about climate change impacts on agriculture . ed militancy: i'm not a big fan of climate change, but i think it's a good idea .", 'a paragraph about carbon emissions and policy changes . another paragraph about policy changes and carbon emissions . a second paragraph about . carbon emissions, policy changes...'] 



# -------------------  RAG Demos ---------------------


In [ ]:
def rag_demo_a(docs=SAMPLE_DOCS):
    """
    Demonstrates a simple RAG (Retrieval-Augmented Generation) workflow 
    by summarizing multiple documents and generating an overall summary using LLM.
    
    Args:
        docs (list): List of document texts to summarize. Defaults to SAMPLE_DOCS.
    
    Returns:
        None: Prints the LLM-generated overall summary to console.
    """
    if not docs:
        print("[Theorem14 RAG Demo A] No docs.")
        return
    sums = summarizer.summarize_batch(docs, model_key="SRSWTI-LW1", max_length=40)
    combined = "\n".join(sums)
    prompt = f"Summaries:\n{combined}\nGive an overall summary."
    resp = call_groq_llm(prompt)
    print("[Theorem14 RAG Demo A] LLM:\n", resp, "\n")

def rag_demo_b(user_query, docs=SAMPLE_DOCS):
    """
    Demonstrates a query-focused RAG workflow by summarizing a randomly selected document
    and using it to answer the user's specific query with an LLM.
    
    Args:
        user_query (str): The user's question or request to be answered
        docs (list): Pool of document texts to select from. Defaults to SAMPLE_DOCS.
    
    Returns:
        None: Prints the LLM-generated response to the user query based on document content.
    """
    if not docs:
        print("[Theorem14 RAG Demo B] No docs.")
        return
    doc = random.choice(docs)
    summ = summarizer.summarize_text(doc, model_key="SRSWTI-LW1", max_length=40)
    prompt = (
        f"User query: {user_query}\nDoc summary:\n{summ}\n"
        "Answer the user's query based on this summary."
    )
    resp = call_groq_llm(prompt)
    print("[Theorem14 RAG Demo B] LLM:\n", resp, "\n")


In [ ]:
rag_demo_a(SAMPLE_DOCS)


[SRSWTI INFO] 2025-03-10 20:58:01 - Starting batch summarization with SRSWTI-LW1
[SRSWTI METRICS] Operation: Batch summarization - Duration: 0.57 seconds
[Theorem14 RAG Demo A] LLM:
 [Groq LLM] Here's a summary of the given topics:

Recent advancements in technology have led to the development of innovative computational models and systems. Neural networks, inspired by the human brain, are composed of interconnected nodes that process and transmit information. Additionally, Generative AI is being used to create unique digital assets, such as NFTs, which can be verified through smart contracts. Meanwhile, quantum computers, which utilize qubits that can exist in multiple states simultaneously, have the potential to perform complex calculations at an exponentially faster rate than classical computers. These breakthroughs are transforming various fields, from artificial intelligence to digital ownership and computing. 



In [ ]:
rag_demo_b("Explain GEN AI in Blockchain", SAMPLE_DOCS)


[SRSWTI INFO] 2025-03-10 20:58:35 - Starting single text summarization with SRSWTI-LW1
[SRSWTI METRICS] Operation: Single text summarization - Duration: 0.45 seconds
[Theorem14 RAG Demo B] LLM:
 [Groq LLM] GEN AI in Blockchain refers to the integration of Generative AI (GEN AI) with blockchain technology. This integration enables the creation of unique digital assets, such as AI-generated art, that can be minted as Non-Fungible Tokens (NFTs). 

The use of blockchain smart contracts allows for the verification of the authenticity and provenance of these AI-generated digital assets. This means that the ownership, origin, and history of the digital assets can be securely tracked and validated on the blockchain, ensuring their legitimacy and value.

In essence, GEN AI in blockchain combines the creative potential of generative AI with the secure and decentralized nature of blockchain, opening up new possibilities for digital art, collectibles, and other unique digital assets. 

